In [1]:
import xarray as xr
import numpy as np

#### function to regrid CCI landcover masks for agriculture to GLASS LAI, keeping track of crop percentages within larger gridcells

##### Because the gridcells of ESA CCI and GLASS are aligned, i.e. GLASS LAI lat/lon is 18 times the lat/lon size of ESA CCI  landcover, we can use the coarsen method to aggregate the gridcells. Both datasets have the center of the gridcell as reference coordinate.

In [2]:
# user settings
years   = np.arange(1992,2015)
basedir = f'/base/dir/'

In [3]:
def CCI_regrid_GLASS_agri(cci_ds, glass_ds):
    ''' function to aggregate cci agricultural classes to glass lat/lon.
        The ouput masks contain coverage (%) of the seperate agricultural classes.'''
    
    lat_refactor    =  int(len(cci_ds.lat.values)/len(glass_ds.lat.values))
    lon_refactor    =  int(len(cci_ds.lon.values)/len(glass_ds.lon.values))
    total_grids     = lat_refactor * lon_refactor

    cci_class_lib   = {'cropland_rainfed': 10, 'cropland_herbs':11, 'cropland_irrigated':20}

    #mask
    #rainfed_mask    = xr.where(lccs_class == cci_class_lib['cropland_rainfed'],1,0)
    rainfed_classes = [cci_class_lib['cropland_rainfed'], cci_class_lib['cropland_herbs']]
    rainfed_mask = xr.where(lccs_class.isin(rainfed_classes), 1, 0)
    irrigated_mask  = xr.where(lccs_class == cci_class_lib['cropland_irrigated'],1,0)

    #regrid
    rainfed_mask_coarse   = rainfed_mask.coarsen(lat=lat_refactor, lon=lon_refactor, boundary='trim').sum()
    irrigated_mask_coarse = irrigated_mask.coarsen(lat=lat_refactor, lon=lon_refactor, boundary='trim').sum()

    # get percentage and output to netcdf
    rainfed_mask_percentage = ((rainfed_mask_coarse / total_grids) * 100).round(1)
    irrigated_mask_percentage = ((irrigated_mask_coarse / total_grids) * 100).round(1)
    total_agriculture_percentage = rainfed_mask_percentage + irrigated_mask_percentage

    # Combine into a dataset
    agriculture_mask = xr.Dataset({
        "rainfed": rainfed_mask_percentage,
        "irrigated": irrigated_mask_percentage,
        "total": total_agriculture_percentage,
    })
    
    # Optionally add metadata
    agriculture_mask.rainfed.attrs["long_name"] = "Rainfed agriculture percentage"
    agriculture_mask.irrigated.attrs["long_name"] = "Irrigated agriculture percentage"
    agriculture_mask.total.attrs["long_name"] = "Total agriculture percentage"
    
    return agriculture_mask

In [4]:
def CCI_regrid_GLASS_nonveg(cci_ds, glass_ds):
    ''' function to aggregate cci non vegetated classes to glass lat/lon.
        The ouput masks contain coverage (%) of the seperate classes.'''
    
    lat_refactor  =  int(len(cci_ds.lat.values)/len(glass_ds.lat.values))
    lon_refactor  =  int(len(cci_ds.lon.values)/len(glass_ds.lon.values))
    total_grids   = lat_refactor * lon_refactor

    cci_class_lib = {'urban':190, 'water':210}

    #mask
    water_mask    = xr.where(lccs_class == cci_class_lib['water'],1,0)
    urban_mask    = xr.where(lccs_class == cci_class_lib['urban'],1,0)

    #regrid
    water_mask_coarse = water_mask.coarsen(lat=lat_refactor, lon=lon_refactor, boundary='trim').sum()
    urban_mask_coarse = urban_mask.coarsen(lat=lat_refactor, lon=lon_refactor, boundary='trim').sum()

    # get percentage and output to netcdf
    water_mask_percentage = ((water_mask_coarse / total_grids) * 100).round(1)
    urban_mask_percentage = ((urban_mask_coarse / total_grids) * 100).round(1)

    # Combine into a dataset
    nonveg_mask = xr.Dataset({
        "water": water_mask_percentage,
        "urban": urban_mask_percentage
    })
    
    # Optionally add metadata
    nonveg_mask.water.attrs["long_name"] = "water bodies percentage"
    nonveg_mask.urban.attrs["long_name"] = "urban areas percentage"
    
    return nonveg_mask

In [ ]:
# apply masks function, loop over datasets
for year in years:
    esa_cci_landcover = xr.open_dataset(f'{basedir}LAI_data/landcover_esa_cci_300m/ESACCI-LC-L4-LCCS-Map-300m-P1Y-{year}-v2.0.7cds.nc')
    lccs_class        = esa_cci_landcover['lccs_class']
    glass_lai_grid    = xr.open_dataset(f'{basedir}LAI_data/GLASS_LAI_netcdf/LAI_{year}.nc')
    nc_mask           = CCI_regrid_GLASS_agri(lccs_class, glass_lai_grid)
    
    # Save to NetCDF
    nc_mask.to_netcdf(f'{basedir}LAI_data/agriculture_mask_{year}.nc')